<a href="https://colab.research.google.com/github/tino926/new_ai_scripts/blob/main/suno_bark/bark_demo_long_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

this modified from suno's sample


## Install

In [ ]:
# install bark (make sure you have torch>=2 for much faster flash-attention)
!pip install git+https://github.com/suno-ai/bark.git

## Basics

In [ ]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from bark.api import semantic_to_waveform
from IPython.display import Audio
from scipy.io.wavfile import write as write_wav
import nltk  # we'll use this to split into sentences
import numpy as np

from bark.generation import (
    generate_text_semantic,
    preload_models,
)

preload_models()
nltk.download('punkt')

In [ ]:
from scipy.io.wavfile import write as write_wav
import nltk  # we'll use this to split into sentences
import numpy as np

# The text to be converted to voice
script = """
Why did the scarecrow win an award? Because he was outstanding in his field!
""".replace("\n", " ").strip()

# splite the text into multiple sentences (strings)
sentences = nltk.sent_tokenize(script)

# indicate the voice, v2/en_speaker_9 is a female English voice
SPEAKER = "v2/en_speaker_9"
GEN_TEMP = 0.6
silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence


pieces = []
for sentence in sentences:
    semantic_tokens = generate_text_semantic(
        sentence,
        history_prompt=SPEAKER,
        temp=GEN_TEMP,
        min_eos_p=0.05,  # this controls how likely the generation is to end
    )

    audio_array = semantic_to_waveform(semantic_tokens, history_prompt=SPEAKER,)
    pieces += [audio_array, silence.copy()]


audio_array = np.concatenate(pieces)


write_wav("bark_generation.wav", SAMPLE_RATE, audio_array)
Audio(audio_array, rate=SAMPLE_RATE*1.1)